# Retriever
- 비정형 질의(query)를 입력 받아 Vector store에서 관련된 내용을 검색하는 기능을 제공하는 인터페이스로 다양한 데이터 소스에서 정보를 검색하여 대규모 언어 모델(LLM) 기반 애플리케이션의 **정확성을** 향상시키는 데 핵심적인 역할을 한다.

![RAG](figures/rag2.png)

## 주요 특징
- **다양한 데이터 소스 지원**
	- Retriever는 벡터 스토어, 그래프 데이터베이스, 관계형 데이터베이스 등 여러 종류의 검색 시스템과 상호작용할 수 있는 통일된 인터페이스를 제공.
- **간단한 인터페이스**: Retriever는 문자열 형태의 쿼리를 입력받아 관련 문서의 리스트를 반환한다. 이러한 단순한 구조 덕분에 다양한 검색 시스템과 쉽게 통합할 수 있다. 

## 다양한 Retriever 유형
   - **벡터 스토어(Vector Store) Retriever**
		- 텍스트 조각마다 임베딩을 생성하여 유사도 검색을 수행합니다. 빠르고 간단한 검색 시스템을 구축할 때 적합하다.
   - **ParentDocument Retriever**
		- 문서를 여러 청크로 나누어 인덱싱한 후, 가장 유사한 청크를 찾아 전체 원본 문서를 반환한다. 작은 정보 조각들이 모여 하나의 문서를 구성할 때 유용하다.
   - **멀티 벡터(Multi Vector) Retriever**
		- 각 문서에 대해 요약이나 가상의 질문 생성과 수동 추가 방식을 통해 문서당 여러 벡터를 생성하여 인덱싱한다. 텍스트 자체보다 더 관련성 있는 정보를 추출할 수 있을 때 사용한다.
   - **Self Query Retriever**
		- LLM을 활용하여 사용자 입력을 텍스트 검색어와 메타데이터 필터로 변환한다. 문서의 메타데이터에 대한 질문을 처리할 때 효과적.
   - **Contextual Compression Retriever**
		- 검색된 문서에서 불필요한 정보를 제거하고, 쿼리와 관련된 핵심 내용만을 추출한다. 

## **고급 검색 패턴 지원**:
   - **앙상블(Ensemble) Retriever**: 여러 Retriever의 검색 결과를 결합하여 더 정확한 결과를 제공한다.
   - **소스 문서 보존(Source Document Retention)**: 인덱싱 과정에서 변환된 문서와 원본 문서 간의 연결을 유지하여, 검색 시 원본 문서를 반환할 수 있게 한다. 

> **인덱싱**
> - 인덱싱은 벡터화된 문서들을 효율적으로 저장하고 관리하여, 유사성 검색 시 빠르게 원하는 정보를 찾을 수 있도록 하는 과정을 말한다. 이를 통해 벡터화된 데이터의 신속하게 찾을 수있다.

# TODO 다음을 작성한다.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache


collection_name = "olmpic_docs"
persist_directory = "vector_store/chroma/olympic"

# Text Loading


# Split


# Vector Store 생성


# Retriever 생성 - "mmr" 방식



In [ ]:
# Prompt Template 생성
messages = [
        ("ai", """
        You are a helpful assistanct. Answer question using only the following context. 
        If you don't know the answer, just say you don't know. Don't make it up. 
        Answer in Korean.
        
        {context}")""",
        ("human", "{question}"),
    ]


# Chain 구성





In [ ]:
# Chain을 이용해 질의




## Map Reduce 방식

- RAG에서 질문과 유사한 document들을 찾아  그대로 질문과 함께 전달하는 방식을 **stuff** 방식이라고 한다.
- Map reduce 방식은 document들에서 질문을 답하는데 적합한 document를 llm을 이용해 찾은 뒤 전달하는 방식이다.
  1. vectorstore에서 질문과 유사한 document들을 유사도 검색으로 찾음
     - 이것들은 단순 유사도라서 질문과 직접 관계 없는 것도 있다.
  2. document들이 질문을 답하는데 적합한지 llm에게 질문해서 관련있는 것들만 문자열로 묶어준다.
  3. 질문과 `2`에서 찾은 내용을 context로해서 llm에 질문하여 최종 답을 받는다.
- context를 좀더 질문과 관련된 내용으로 전달 할 수있는 장점이 있는 반면 비용이 드는 폐쇄형 llm을 사용하는 경우 비용이 추가로 드는 단점이 있다.

In [ ]:
map_doc_prompt = ChatPromptTemplate.from_messages([
    ("system",  """
Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text verbatim. If there is no relevant text, return : ''
-------
{context}
"""),
    ("human", "{question}"),
])


In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)